In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import random
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.datasets import make_classification

pd.options.display.max_rows = 6
pd.options.display.max_columns = 300

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_sample = pd.read_csv('sample_submission.csv')
df_waves = pd.read_csv('waves.csv')

In [3]:
#Подготовка данных под единый вид, удобный для анализа и работы с моделями
#Решение про ряд условий np.where не самое элегантное, но при этом рабочее
#

df_test['Native city+'] = df_test['Native city'].apply(lambda x: str(x).upper())

df_test['Native city+'] = df_test['Native city+'].apply(lambda x: str(x)\
                                                      .replace('Ё', 'Е').replace('Г.', '').replace('ПОС.', ''))

df_test['Native city+'] = df_test['Native city+'].apply(lambda x: str(x).replace('С. ', '').replace('ГОР.', ''))
df_test['Native city+'] = df_test['Native city+'].apply(lambda x: str(x).replace('СТ. ', '').replace('П.', ''))
df_test['Native city+'] = df_test['Native city+'].apply(lambda x: str(x).replace('ДЕР. ', '').replace('Д. ', ''))

df_test['Native city+'] = np.where((df_test['Native city+'].apply(lambda x: str(x)[-1:]) == ' '),
                                  (df_test['Native city+'].apply(lambda x: str(x)[:-1])),
                                  df_test['Native city+'])
df_test['Native city+'] = np.where((df_test['Native city+'].apply(lambda x: str(x)[:1]) == ' '),
                                  (df_test['Native city+'].apply(lambda x: str(x)[1:])),
                                  df_test['Native city+'])

df_test['Birth date+'] = pd.to_datetime(df_test['Birth date'])

df_test['Birth_date_category'] = np.where((df_test['Birth date+'] < dt.date(1980, 1, 1)), '40+',
                                          (np.where(((df_test['Birth date+'] > dt.date(1980, 1, 1)) &
                                                     (df_test['Birth date+'] < dt.date(1985, 1, 1))), '35-40',
                                                    (np.where(((df_test['Birth date+'] > dt.date(1985, 1, 1)) &
                                                               (df_test['Birth date+'] < dt.date(1990, 1, 1))), '35-30',
                                                              (np.where(((df_test['Birth date+'] > dt.date(1990, 1, 1)) &
                                                                         (df_test['Birth date+'] < dt.date(1995, 1, 1))), '30-25',
                                                                        (np.where(((df_test['Birth date+'] > dt.date(1995, 1, 1)) &
                                                                                   (df_test['Birth date+'] < dt.date(1998, 1, 1))), '25-21',
                                                                                  (np.where(((df_test['Birth date+'] > dt.date(1998, 1, 1)) &
                                                                                             (df_test['Birth date+'] < dt.date(2001, 1, 1))), '21-18', '-18')))))))))))

df_test['Level_int'] = df_test['Level'].apply(lambda x: int(x))

df_train['Native city+'] = df_train['Native city'].apply(lambda x: str(x).upper())

df_train['Native city+'] = df_train['Native city+'].apply(lambda x: str(x)\
                                                      .replace('Ё', 'Е').replace('Г.', '').replace('ПОС.', ''))

df_train['Native city+'] = df_train['Native city+'].apply(lambda x: str(x).replace('С. ', '').replace('ГОР.', ''))
df_train['Native city+'] = df_train['Native city+'].apply(lambda x: str(x).replace('СТ. ', '').replace('П.', ''))
df_train['Native city+'] = df_train['Native city+'].apply(lambda x: str(x).replace('ДЕР. ', '').replace('Д. ', ''))

df_train['Native city+'] = np.where((df_train['Native city+'].apply(lambda x: str(x)[-1:]) == ' '),
                                  (df_train['Native city+'].apply(lambda x: str(x)[:-1])),
                                  df_train['Native city+'])
df_train['Native city+'] = np.where((df_train['Native city+'].apply(lambda x: str(x)[:1]) == ' '),
                                  (df_train['Native city+'].apply(lambda x: str(x)[1:])),
                                  df_train['Native city+'])

df_train['Birth date+'] = pd.to_datetime(df_train['Birth date'])

df_train['Birth_date_category'] = np.where((df_train['Birth date+'] < dt.date(1980, 1, 1)), '40+',
                                          (np.where(((df_train['Birth date+'] > dt.date(1980, 1, 1)) &
                                                     (df_train['Birth date+'] < dt.date(1985, 1, 1))), '35-40',
                                                    (np.where(((df_train['Birth date+'] > dt.date(1985, 1, 1)) &
                                                               (df_train['Birth date+'] < dt.date(1990, 1, 1))), '35-30',
                                                              (np.where(((df_train['Birth date+'] > dt.date(1990, 1, 1)) &
                                                                         (df_train['Birth date+'] < dt.date(1995, 1, 1))), '30-25',
                                                                        (np.where(((df_train['Birth date+'] > dt.date(1995, 1, 1)) &
                                                                                   (df_train['Birth date+'] < dt.date(1998, 1, 1))), '25-21',
                                                                                  (np.where(((df_train['Birth date+'] > dt.date(1998, 1, 1)) &
                                                                                             (df_train['Birth date+'] < dt.date(2001, 1, 1))), '21-18', '-18')))))))))))

df_train['Level_int'] = df_train['Level'].apply(lambda x: int(x))

In [4]:
dict_obl = pd.read_excel('Goroda res +.xlsx')

In [5]:
df_train = df_train.merge(dict_obl, on='Native city+').drop_duplicates('id')
df_test = df_test.merge(dict_obl, on='Native city+').drop_duplicates('id')

df_train['Year'] = df_train['Birth date+'].apply(lambda x: x.year)
df_test['Year'] = df_test['Birth date+'].apply(lambda x: x.year)

df_train['Gender_int'] = df_train['Gender'].apply(lambda x: int(str(x)\
                                                                .replace('female', '0')\
                                                                .replace('male', '1')))
df_test['Gender_int'] = df_test['Gender'].apply(lambda x: int(str(x)\
                                                              .replace('female', '0')\
                                                              .replace('male', '1')))

In [6]:
df_test['Life status_int'] = df_test['Life status'].apply(lambda x: int(str(x).replace('nan', '0')\
                                                                        .replace('work', '1')\
                                                                        .replace('study', '2')\
                                                                        .replace('other', '3')))
df_train['Life status_int'] = df_train['Life status'].apply(lambda x: int(str(x).replace('nan', '0')\
                                                                        .replace('work', '1')\
                                                                        .replace('study', '2')\
                                                                        .replace('other', '3')))

In [7]:
df_test['Birth_date_category_int'] = df_test['Birth_date_category'].apply(lambda x: int(x\
                                                .replace('21-18', '1')\
                                                .replace('25-21', '2')\
                                                .replace('30-25', '3')\
                                                .replace('35-30', '4')\
                                                .replace('35-40', '5')\
                                                .replace('40+', '6')\
                                                .replace('-18', '0')))
df_train['Birth_date_category_int'] = df_train['Birth_date_category'].apply(lambda x: int(x\
                                                .replace('21-18', '1')\
                                                .replace('25-21', '2')\
                                                .replace('30-25', '3')\
                                                .replace('35-30', '4')\
                                                .replace('35-40', '5')\
                                                .replace('40+', '6')\
                                                .replace('-18', '0')))

In [8]:
df_train['Contract termination date+'] = df_train['Contract termination date'].fillna(0)#

In [9]:
df_train['Contract termination date+'] = df_train['Contract termination date+'].apply(lambda x: (str(x)[:10]))

In [10]:
df_train['contract_status'] = df_train['contract_status'].apply(lambda x: float(x))

In [11]:
# df_train['Birth_date_category_int'] = df_train['Birth_date_category_int'].apply(lambda x: float(x))
# df_train["Wave id"] = df_train["Wave id"].apply(lambda x: float(x))
# df_train['Gender_int'] = df_train['Gender_int'].apply(lambda x: float(x))
# df_train['Москва/НеМосква'] = df_train['Москва/НеМосква'].apply(lambda x: float(x))

# df_test['Birth_date_category_int'] = df_train['Birth_date_category_int'].apply(lambda x: float(x))
# df_test["Wave id"] = df_train["Wave id"].apply(lambda x: float(x))
# df_test['Gender_int'] = df_train['Gender_int'].apply(lambda x: float(x))
# df_test['Москва/НеМосква'] = df_train['Москва/НеМосква'].apply(lambda x: float(x))

In [12]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [13]:
#Пробовали использовать большое количество фичей, но результат было сложно предугодать.
#Оставили наиболее подходящее количество, даже с учетом что подговтовили намного больше вариаций.
y = df_train["contract_status"]

features = ["Level", 'Wave id', 'Gender_int', 'exam_final', 'Москва/НеМосква']
X = pd.get_dummies(df_train[features])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

X_test = pd.get_dummies(df_test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=3)
model.fit(X, y)
predictions = model.predict(X_test)

# accuracy_score(y_test, predictions), precision_score(y_test, predictions), recall_score(y_test, predictions)

In [14]:
output = pd.DataFrame({'id': df_test.id, 'contract status': predictions})
output = output.rename(columns={'contract status': 'contract_status'})
output.to_csv('test1.csv', index=False, sep=',')

In [15]:
print('Отчисленные', len(output[((output['contract_status'] == 0))]))
print('Оставшиеся', len(output[((output['contract_status'] == 1))]))

Отчисленные 28
Оставшиеся 678


In [16]:
# Мы не успели подогнать решение под float, не дочитали ряд статей, так что испытывали небольшой метод рандома
output['contract_status'] = output['contract_status'].apply(lambda x: float(str(x)\
                                                                            .replace('1.0', '0.96')\
                                                                            .replace('0.0', '0.17') +\
                                                                            (str(random.random())[3:])))

In [18]:
output.to_csv('test123.csv', index=False)

In [19]:
pd.read_csv('test123.csv')

,id,contract_status
0,685512434,0.964335
1,1443818764,0.961362
2,157712083,0.965679
...,...,...
703,1558618271,0.962178
704,691754591,0.963158
705,889605876,0.964931
